<a href="https://colab.research.google.com/github/carolflyjs/cs230/blob/master/Data_Processing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyspellchecker 

     |████████████████████████████████| 1.9MB 2.8MB/s 


In [0]:
from spellchecker import SpellChecker
import pandas as pd
import os
import tensorflow as tf
import tensorflow_hub as hub
import keras
import numpy as np
from keras.layers import Input, Dense, concatenate, Dot, Embedding, LSTM
from keras.engine import Layer
from keras.models import Model
from keras import backend as K
from keras.preprocessing import text
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

ModuleNotFoundError: ignored

In [0]:
def load_file(path, delimiter=","):
    df = pd.read_csv(path, delimiter=delimiter)
    df = shuffle(df).reset_index(drop=True)
    class_dict = {
      "BT": 0,
      "NT": 1,
      "NPT": 2,
      "PT": 2,
      "RT": 3,
      "URT": 3
    }
    df["label"] = df["label"].apply(lambda x: class_dict[x])
    return df

def data_prep(df, x_columns, y_columns):
    X_train = df[x_columns]
    Y_train = df[y_columns]
    Y_train = keras.utils.to_categorical(Y_train, num_classes=4)
    return X_train, Y_train


In [0]:
def removeSpellingErrors(df):
    new_list = []
    spell = SpellChecker()
    raw_df_list = df.values.tolist()

    for (source, target, label) in raw_df_list:
        if len(spell.unknown(source.split())) == 0 and len(spell.unknown(target.split())) == 0:
            new_list.append((source, target, label))
    new_df = pd.DataFrame(new_list, columns=["source", "target", "label"])
    return new_df

In [0]:
def tokenize_and_pad(X_train):
    def tokenize(X_train):
        t = text.Tokenizer()
        fit_text = pd.concat([X_train["source"], X_train["target"]])
        t.fit_on_texts(fit_text)
        X_train_transformed = X_train.copy()
        X_train_transformed["source"] = t.texts_to_sequences(X_train["source"])
        X_train_transformed["target"] = t.texts_to_sequences(X_train["target"])
        max_value = 0
        for series in ["source", "target"]:
            current_max = (df[series].apply(lambda ls: len(ls))).max()
            if current_max > max_value:
                max_value = current_max
        max_value = max_value
        vocab_size = len(t.index_word) + 1
        return X_train_transformed,  max_value, vocab_size, t
    def pad(X, max_value):
        source = sequence.pad_sequences(X["source"],  maxlen=max_value)
        target = sequence.pad_sequences(X["target"],  maxlen=max_value)
        return pd.DataFrame(data=np.concatenate((source, target), axis=1))
    X_train_tokenized, max_value, vocab_size, tokenizer = tokenize(X_train)
    X_train_padded = pad(X_train_tokenized, max_value=max_value)
    return X_train_padded, max_value, vocab_size, tokenizer

In [0]:
raw_df = load_file("/content/drive/My Drive/cs230/train.csv")
df = removeSpellingErrors(raw_df)
X_train_raw, Y_train = data_prep(df, ["source", "target"], ["label"])
X_train_tokenized, max_value, vocab_size, tokenizer = tokenize_and_pad(X_train_raw)

In [0]:
df.groupby("label").count()

,source,target
label,,
0,2727,2727
1,2638,2638
2,11643,11643
3,5880,5880


In [0]:
class_weight = {0: 6, 1: 6, 2: 1, 3: 2}